# 🚀 Framework de RL para Previsão de Ciclos Econômicos

**Framework avançado que combina Reinforcement Learning com modelos supervisionados para previsão de séries temporais.**

Este notebook demonstra o uso completo no Google Colab.

---

## 📋 Conteúdo
1. Setup e Instalação
2. Geração de Dados
3. Criação de Modelos Base
4. Ambiente de RL
5. Treinamento do Agente
6. Avaliação e Resultados
7. Visualizações

## 1️⃣ Setup e Instalação

Clone o repositório e instale dependências (leva ~2 minutos).

In [ ]:
# Clone repositório
!git clone https://github.com/cbaracho200/Previs-o-ciclos-Econ-mico.git
%cd Previs-o-ciclos-Econ-mico

# Instala dependências
!pip install -q torch gymnasium statsmodels pmdarima xgboost pandas numpy matplotlib seaborn tqdm scikit-learn

print("\n✅ Instalação concluída!")

### Imports e Configuração

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Configuração de estilo
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)

# Adiciona src ao path
sys.path.insert(0, '/content/Previs-o-ciclos-Econ-mico')

# Imports do framework
from src.utils.data_utils import generate_synthetic_data, split_data
from src.models.arima_model import ARIMAPredictor
from src.models.lstm_model import LSTMPredictor
from src.models.xgboost_model import XGBoostPredictor
from src.models.ensemble_predictor import EnsemblePredictor
from src.environments.timeseries_env import TimeSeriesEnv
from src.agents.rl_agent import RLAgent
from src.training.trainer import RLTrainer
from src.utils.visualization import plot_predictions, plot_coefficients
from src.utils.metrics import calculate_metrics

print("✅ Imports realizados com sucesso!")

# Tenta importar agente avançado
try:
    from src.agents.rl_agent_advanced import AdvancedRLAgent
    from src.training.trainer_advanced import AdvancedRLTrainer
    print("✅ Agente Avançado (PhD) disponível!")
    ADVANCED_AVAILABLE = True
except ImportError as e:
    print(f"⚠️  Agente Avançado não disponível (use RLAgent padrão)")
    print(f"   Erro: {e}")
    ADVANCED_AVAILABLE = False

## 2️⃣ Geração de Dados

Gera dados sintéticos de séries temporais com tendência, sazonalidade e ruído.

In [ ]:
# Gera dados sintéticos
print("Gerando dados sintéticos...")
data = generate_synthetic_data(
    n_points=300,
    trend=0.05,
    seasonality_amplitude=15.0,
    seasonality_period=12,
    noise_std=3.0,
    seed=42
)

# Divide dados
train_data, val_data, test_data = split_data(
    data, 
    train_ratio=0.7, 
    val_ratio=0.15
)

print(f"✅ Dados gerados: {len(data)} pontos")
print(f"   • Treino: {len(train_data)} pontos")
print(f"   • Validação: {len(val_data)} pontos")
print(f"   • Teste: {len(test_data)} pontos")

# Visualiza dados
plt.figure(figsize=(14, 6))
plt.plot(data['date'], data['value'], linewidth=2, label='Dados Completos', alpha=0.8)
plt.axvline(x=train_data['date'].iloc[-1], color='red', linestyle='--', 
            linewidth=2, label='Fim Treino', alpha=0.7)
plt.axvline(x=val_data['date'].iloc[-1], color='orange', linestyle='--', 
            linewidth=2, label='Fim Validação', alpha=0.7)
plt.xlabel('Data', fontsize=12)
plt.ylabel('Valor', fontsize=12)
plt.title('Série Temporal Sintética - Divisão dos Dados', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Estatísticas descritivas
print("\n📊 Estatísticas dos dados:")
print(data['value'].describe())

## 3️⃣ Criação de Modelos Base

Cria três modelos supervisionados:
- **ARIMA**: Captura tendências lineares e sazonalidades
- **LSTM**: Captura dependências de longo prazo
- **XGBoost**: Captura relações não-lineares

In [ ]:
print("🔧 Criando modelos base...\n")

models = [
    ARIMAPredictor(
        order=(2, 1, 2), 
        name="ARIMA"
    ),
    LSTMPredictor(
        lookback=12, 
        hidden_size=32, 
        num_layers=2, 
        epochs=30,  # Reduzido para Colab
        batch_size=16,
        name="LSTM"
    ),
    XGBoostPredictor(
        lookback=12, 
        n_estimators=50,
        max_depth=4,
        name="XGBoost"
    )
]

print(f"✅ {len(models)} modelos criados:")
for model in models:
    print(f"   • {model.name}")

### Criação e Treinamento do Ensemble

In [ ]:
print("🔄 Criando e treinando ensemble...\n")

ensemble = EnsemblePredictor(models)
ensemble.fit(train_data['value'])

print("\n✅ Ensemble criado e treinado!")
print(f"\n📊 Pesos iniciais (iguais): {ensemble.get_weights()}")

## 4️⃣ Ambiente de RL

Cria o ambiente Gymnasium customizado para otimização de séries temporais.

In [ ]:
print("🌍 Criando ambiente de RL...\n")

FORECAST_HORIZON = 6  # Previsão de 6 meses à frente
WINDOW_SIZE = 24      # Janela de observação de 24 pontos

env = TimeSeriesEnv(
    data=train_data,
    forecast_horizon=FORECAST_HORIZON,
    window_size=WINDOW_SIZE,
    n_coefficients=len(models),
    max_steps=50
)

print(f"✅ Ambiente criado com sucesso!")
print(f"\n📋 Configuração:")
print(f"   • Horizonte de previsão: {env.forecast_horizon} meses")
print(f"   • Janela de observação: {env.window_size} pontos")
print(f"   • Número de coeficientes: {env.n_coefficients}")
print(f"   • Dimensão do estado: {env.observation_space.shape[0]}")
print(f"   • Dimensão da ação: {env.action_space.shape[0]}")

### Criação do Agente RL (PPO)

In [ ]:
print("🤖 Criando agente RL (PPO)...\n")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]

agent = RLAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    learning_rate=3e-4,
    gamma=0.99,
    gae_lambda=0.95,
    clip_epsilon=0.2,
    hidden_dim=128,
    device='cpu'  # Mude para 'cuda' se tiver GPU habilitada
)

print(f"✅ Agente PPO criado!")
print(f"\n🧠 Arquitetura:")
print(f"   • Dimensão do estado: {state_dim}")
print(f"   • Dimensão da ação: {action_dim}")
print(f"   • Camadas ocultas: 128 neurônios")
print(f"   • Algoritmo: Proximal Policy Optimization (PPO)")

### ⚠️ IMPORTANTE: Use as Dimensões Corretas!

**ERRO COMUM**: Nunca use valores hardcoded como `state_dim=44` ou `action_dim=10`!

**CORRETO**: Sempre pegue do ambiente:

## 5️⃣ Treinamento do Agente

⏰ **Atenção**: Esta célula pode levar 5-10 minutos no Colab gratuito.

O agente aprenderá os melhores coeficientes para combinar os modelos.

In [ ]:
# 💡 OPCIONAL: Use este código para treinar com o agente avançado
# Descomente para usar (requer ~2-3GB RAM adicional)

if ADVANCED_AVAILABLE:
    print("🎓 Treinando com Agente Avançado (PhD)...\n")
    print("⏰ Tempo estimado: 10-15 minutos no Colab gratuito\n")
    
    # Cria agente avançado
    advanced_agent = AdvancedRLAgent(
        state_dim=env.observation_space.shape[0],  # ✅ CORRETO: pega do ambiente
        action_dim=env.action_space.shape[0],      # ✅ CORRETO: pega do ambiente
        learning_rate=1e-4,
        hidden_dim=256,        # Reduzido para Colab (use 512 com GPU)
        num_heads=4,           # Reduzido para Colab (use 8 com GPU)
        num_layers=2,          # Reduzido para Colab (use 3 com GPU)
        use_per=True,
        use_noisy=True,
        use_lstm=True,
        device='cpu'           # Mude para 'cuda' se tiver GPU
    )
    
    # Cria trainer avançado
    advanced_trainer = AdvancedRLTrainer(
        env, 
        advanced_agent, 
        ensemble,
        use_curriculum=True
    )
    
    # Treina
    advanced_history = advanced_trainer.train(
        n_episodes=100,        # Use 200-500 em produção
        max_steps=50,
        eval_frequency=25,
        early_stopping=True,
        verbose=True
    )
    
    # Substitui trainer e agent pelas versões avançadas
    trainer = advanced_trainer
    agent = advanced_agent
    history = advanced_history
    
    print("\n" + "="*80)
    print("🎉 TREINAMENTO AVANÇADO CONCLUÍDO!")
    print("="*80)
else:
    print("⚠️  Agente Avançado não disponível. Use o treinamento padrão acima.")

## 🎓 ALTERNATIVA: Agente Avançado (Nível PhD)

**Quer resultados ainda melhores?** Use o AdvancedRLAgent com técnicas state-of-the-art!

### Características:
- ✅ Transformer-based Actor-Critic
- ✅ Multi-Head Attention (8 heads)
- ✅ Prioritized Experience Replay (PER)
- ✅ Noisy Networks (exploração adaptativa)
- ✅ LSTM Memory
- ✅ Adaptive Entropy Regularization
- ✅ Curriculum Learning

**⚠️ Requer mais RAM e tempo de treinamento**

In [ ]:
print("🎯 Iniciando treinamento do agente RL...\n")
print("⏰ Tempo estimado: 5-10 minutos no Colab gratuito\n")

trainer = RLTrainer(env, agent, ensemble)

# Treinamento (reduzido para Colab)
history = trainer.train(
    n_episodes=100,      # Use 200-500 em produção
    max_steps=50,
    eval_frequency=25,
    save_frequency=50,
    verbose=True
)

print("\n" + "="*80)
print("🎉 TREINAMENTO CONCLUÍDO!")
print("="*80)

## 6️⃣ Avaliação e Resultados

Avalia o agente treinado e extrai os melhores coeficientes.

In [ ]:
print("📊 Avaliando modelo treinado...\n")

# Avaliação
eval_results = trainer.evaluate(
    n_episodes=10, 
    deterministic=True, 
    verbose=True
)

# Extrai melhores coeficientes
best_coefficients = trainer.get_best_coefficients()

if best_coefficients is not None:
    print(f"\n{'='*60}")
    print("🏆 MELHORES COEFICIENTES ENCONTRADOS")
    print(f"{'='*60}")
    for i, (model, coef) in enumerate(zip(models, best_coefficients)):
        print(f"   {model.name:12s}: {coef:.4f} ({coef*100:.1f}%)")
    print(f"{'='*60}\n")
    
    # Atualiza ensemble com melhores coeficientes
    ensemble.update_weights(best_coefficients)
    print("✅ Ensemble atualizado com coeficientes otimizados!")
else:
    print("⚠️ Coeficientes não disponíveis, usando pesos iguais.")

### Teste no Conjunto de Teste

In [ ]:
print("🧪 Testando no conjunto de teste...\n")

# Retreina ensemble com treino + validação
full_train_data = pd.concat([train_data, val_data])
ensemble.fit(full_train_data['value'])

# Faz previsão
forecast_horizon = 12
predictions = ensemble.predict(steps=forecast_horizon)

# Limita ao tamanho dos dados de teste
actual_values = test_data['value'].values[:forecast_horizon]
predictions = predictions[:len(actual_values)]

# Calcula métricas
metrics = calculate_metrics(actual_values, predictions)

# Exibe resultados
print(f"\n{'='*70}")
print(f"{'RESULTADOS NO CONJUNTO DE TESTE':^70}")
print(f"{'='*70}")
print(f"\n📈 Métricas de Precisão:")
print(f"   • MAPE (Mean Absolute % Error): {metrics['mape']:8.2f}%")
print(f"   • RMSE (Root Mean Squared Error): {metrics['rmse']:8.4f}")
print(f"   • MAE  (Mean Absolute Error):     {metrics['mae']:8.4f}")
print(f"   • R²   (Coef. Determinação):      {metrics['r2']:8.4f}")
print(f"   • Acurácia Direcional:            {metrics['directional_accuracy']:8.2f}%")
print(f"\n{'='*70}\n")

# Interpretação
if metrics['mape'] < 5:
    interpretation = "🌟 EXCELENTE!"
elif metrics['mape'] < 10:
    interpretation = "✅ MUITO BOM!"
elif metrics['mape'] < 15:
    interpretation = "👍 BOM!"
else:
    interpretation = "⚠️ ACEITÁVEL (pode melhorar com mais treinamento)"

print(f"💡 Interpretação: {interpretation}")
print(f"\nMAPE < 5% = Excelente | 5-10% = Muito Bom | 10-15% = Bom | >15% = Aceitável")

## 7️⃣ Visualizações

Gera gráficos detalhados dos resultados.

In [ ]:
# Visualização 1: Previsões vs Valores Reais
print("📊 Gerando visualizações...\n")

plot_predictions(
    actual_values,
    predictions,
    title=f"Ensemble Otimizado por RL - Teste (MAPE: {metrics['mape']:.2f}%)"
)

In [ ]:
# Visualização 2: Coeficientes Otimizados
if best_coefficients is not None:
    plot_coefficients(
        best_coefficients,
        model_names=[m.name for m in models],
        title="Coeficientes Otimizados pelo Agente RL (PPO)"
    )
else:
    print("⚠️ Coeficientes não disponíveis para visualização.")

In [ ]:
# Visualização 3: Histórico de Treinamento
trainer.plot_training_progress()

### Comparação: Antes vs Depois do RL

In [ ]:
# Compara ensemble com pesos iguais vs otimizados
print("🔄 Comparando resultados...\n")

# Ensemble com pesos iguais
ensemble_equal = EnsemblePredictor(models, weights=np.ones(len(models)))
ensemble_equal.fit(full_train_data['value'])
predictions_equal = ensemble_equal.predict(steps=len(actual_values))

# Métricas do ensemble com pesos iguais
metrics_equal = calculate_metrics(actual_values, predictions_equal)

# Tabela de comparação
print(f"\n{'='*80}")
print(f"{'COMPARAÇÃO: ENSEMBLE COM PESOS IGUAIS vs OTIMIZADO POR RL':^80}")
print(f"{'='*80}")
print(f"\n{'Métrica':<25} {'Pesos Iguais':>15} {'Otimizado RL':>15} {'Melhoria':>15}")
print(f"{'-'*80}")

for metric_name in ['mape', 'rmse', 'mae', 'r2']:
    equal_val = metrics_equal[metric_name]
    optim_val = metrics[metric_name]
    
    if metric_name == 'r2':
        improvement = ((optim_val - equal_val) / abs(equal_val)) * 100
        print(f"{metric_name.upper():<25} {equal_val:>15.4f} {optim_val:>15.4f} {improvement:>+14.1f}%")
    else:
        improvement = ((equal_val - optim_val) / equal_val) * 100
        print(f"{metric_name.upper():<25} {equal_val:>15.4f} {optim_val:>15.4f} {improvement:>+14.1f}%")

print(f"{'-'*80}")
print(f"\n💡 Valores positivos em 'Melhoria' indicam que o RL otimizou o ensemble!")
print(f"{'='*80}\n")

## 💾 Salvando Resultados

Opcional: Salve o modelo treinado no Google Drive.

In [ ]:
# Descomente para salvar no Google Drive

# from google.colab import drive
# drive.mount('/content/drive')

# # Salva checkpoint
# import os
# save_dir = '/content/drive/MyDrive/rl_forecasting_models'
# os.makedirs(save_dir, exist_ok=True)

# # Salva modelo
# agent.save(f'{save_dir}/agent_model.pt')
# print(f"✅ Modelo salvo em: {save_dir}")

# # Salva coeficientes
# if best_coefficients is not None:
#     np.save(f'{save_dir}/best_coefficients.npy', best_coefficients)
#     print(f"✅ Coeficientes salvos!")

print("\n💡 Descomente o código acima para salvar no Google Drive.")

## 🎯 Próximos Passos

### 1. Use seus próprios dados
```python
from google.colab import files
uploaded = files.upload()
data = pd.read_csv(list(uploaded.keys())[0])
# Certifique-se de ter uma coluna 'value'
```

### 2. Ajuste hiperparâmetros
- Aumente `n_episodes` para melhor treinamento (200-500)
- Ajuste `learning_rate` do agente (1e-4 a 1e-3)
- Modifique `forecast_horizon` (6-12 meses)

### 3. Adicione mais modelos
```python
from statsmodels.tsa.holtwinters import ExponentialSmoothing
# Implemente seu próprio modelo baseado em BasePredictor
```

### 4. Ative GPU no Colab
- `Runtime > Change runtime type > GPU`
- Mude `device='cuda'` no agente

---

## 📚 Documentação Completa

Visite o [README do projeto](https://github.com/cbaracho200/Previs-o-ciclos-Econ-mico) para mais informações!

---

**Desenvolvido com ❤️ para previsão de ciclos econômicos usando Reinforcement Learning**